### 🏎️ F1 코너링 DNA 클러스터링

- 문제 정의
    - "몬자의 파라볼리카(Parabolica)와 스즈카의 130R은 비슷한 코너인가?"
    - "베르스타펜은 고속 코너에 강한가, 저속 헤어핀에 강한가?"
    - 이런 질문의 답을 알기 위해선 F1 캘린더에 존재하는 수백 개의 코너를 **특성별로** **그룹화(Clustering)**해야 합니다.

- 분석 방법 (K-Means Clustering)
    - 데이터: 2018~2025년 전체 서킷의 모든 코너 정보
    - Feature (X):
        1. 코너 최저 속도 (Apex Speed): 해당 코너에서 찍힌 가장 낮은 속도
        2. 코너 각도 (Angle): 코너가 얼마나 급격하게 꺾이는지 (FastF1 메타데이터 활용)
    - 알고리즘: K-Means를 사용하여 코너들을 자동으로 $K$개의 그룹(예: 초고속, 고속, 중속, 저속 헤어핀)으로 분류

In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === [설정] ===
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "f1_analytics_db"
YEARS = range(2018, 2026) # 2018년부터 2025년까지 분석

# 분석할 주요 서킷 리스트
CIRCUITS = [
    "Bahrain", "Saudi Arabia", "Australia", "Japan", "China", "Miami",
    "Imola", "Monaco", "Canada", "Spain", "Austria", "Great Britain",
    "Hungary", "Belgium", "Netherlands", "Monza", "Azerbaijan", "Singapore",
    "Austin", "Mexico", "Brazil", "Las Vegas", "Qatar", "Abu Dhabi"
]

# 서킷 한글 매핑
CIRCUIT_KOR = {
    "Bahrain": "바레인", "Saudi Arabia": "제다", "Australia": "멜버른",
    "Japan": "스즈카", "China": "상하이", "Miami": "마이애미",
    "Imola": "이몰라", "Monaco": "모나코", "Canada": "몬트리올",
    "Spain": "바르셀로나", "Austria": "레드불링", "Great Britain": "실버스톤",
    "Hungary": "헝가로링", "Belgium": "스파", "Netherlands": "잔트보르트",
    "Monza": "몬자", "Azerbaijan": "바쿠", "Singapore": "싱가포르",
    "Austin": "오스틴", "Mexico": "멕시코", "Brazil": "인터라고스",
    "Las Vegas": "라스베가스", "Qatar": "루사일", "Abu Dhabi": "야스마리나"
}

print(f"✅ 설정 완료: {len(CIRCUITS)}개 서킷, {len(YEARS)}개년도 분석 준비")

✅ 설정 완료: 24개 서킷, 8개년도 분석 준비


In [3]:
def get_multi_year_corner_data():
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]

    corner_dataset = []
    print(f"🏎️ 대규모 코너 데이터 추출 시작 ({YEARS[0]}-{YEARS[-1]})...")

    for year in YEARS:
        print(f"   📅 Processing {year} Season...")

        for circuit in CIRCUITS:
            # 1. 메타데이터 찾기
            meta = db.races_meta.find_one({
                "Year": year,
                "Circuit": {"$regex": circuit, "$options": "i"},
                "SessionType": "Qualifying"
            })
            if not meta: continue

            # 2. 폴 포지션 랩 찾기
            pole_lap = db.laps.find_one(
                {"Year": year, "Round": meta['Round'], "SessionType": "Qualifying", "IsAccurate": True},
                sort=[("LapTime_Sec", 1)]
            )
            if not pole_lap: continue

            # 3. 텔레메트리 가져오기 (드라이버 매핑 적용)
            try:
                # 드라이버 번호 매핑
                res = db.results.find_one({"Year": year, "Round": meta['Round'], "Driver": pole_lap['Driver']})
                if not res: continue
                drv_num = str(res['DriverNumber'])

                telemetry = list(db.telemetry.find(
                    {"Year": year, "Round": meta['Round'], "SessionType": "Qualifying", "Driver": drv_num},
                    {"Distance": 1, "Speed": 1, "_id": 0}
                ).sort("Distance", 1))

                if not telemetry: continue

                df_tel = pd.DataFrame(telemetry)

                # 4. 코너 감지 (Local Minima 알고리즘)
                # 노이즈 완화를 위한 이동 평균
                df_tel['Speed_Smooth'] = df_tel['Speed'].rolling(window=5, center=True).mean()

                # 변곡점(최저 속도 지점) 찾기
                df_tel['Prev'] = df_tel['Speed_Smooth'].shift(5)
                df_tel['Next'] = df_tel['Speed_Smooth'].shift(-5)

                corners = df_tel[
                    (df_tel['Speed_Smooth'] < df_tel['Prev']) &
                    (df_tel['Speed_Smooth'] < df_tel['Next']) &
                    (df_tel['Speed_Smooth'] > 40) # 40km/h 이상 (너무 느린건 제외)
                ].copy()

                # 중복 제거 (200m 이내 근접 데이터 병합)
                last_dist = -1000
                corner_count = 1

                for _, row in corners.iterrows():
                    if row['Distance'] - last_dist > 200:
                        corner_dataset.append({
                            "Year": year,
                            "Circuit": CIRCUIT_KOR.get(circuit, circuit),
                            "OriginalCircuit": circuit,
                            "CornerID": f"T{corner_count}",
                            "ApexSpeed": row['Speed_Smooth'],
                            "Distance": row['Distance'],
                            "Driver": pole_lap['Driver']
                        })
                        last_dist = row['Distance']
                        corner_count += 1

            except Exception as e:
                continue

    print(f"✅ 총 {len(corner_dataset)}개의 코너 데이터 확보 완료.")
    return pd.DataFrame(corner_dataset)

In [4]:
# 데이터 추출 실행
df_all_corners = get_multi_year_corner_data()

# 결과 확인
if not df_all_corners.empty:
    print("\n🔎 데이터 샘플:")
    display(df_all_corners.head())
else:
    print("❌ 데이터를 추출하지 못했습니다.")

🏎️ 대규모 코너 데이터 추출 시작 (2018-2025)...
   📅 Processing 2018 Season...
   📅 Processing 2019 Season...
   📅 Processing 2020 Season...
   📅 Processing 2021 Season...
   📅 Processing 2022 Season...
   📅 Processing 2023 Season...
   📅 Processing 2024 Season...
   📅 Processing 2025 Season...
✅ 총 13339개의 코너 데이터 확보 완료.

🔎 데이터 샘플:


,Year,Circuit,OriginalCircuit,CornerID,ApexSpeed,Distance,Driver
0,2018,바레인,Bahrain,T1,53.628288,355.358056,VET
1,2018,바레인,Bahrain,T2,90.480995,1146.385000,VET
2,2018,바레인,Bahrain,T3,135.711667,1509.781944,VET
3,2018,바레인,Bahrain,T4,73.115460,1869.712222,VET
4,2018,바레인,Bahrain,T5,67.966169,2335.276187,VET


In [6]:
def perform_clustering(df):
    if df.empty: return pd.DataFrame()

    print("\n🤖 K-Means Clustering 수행 중 (전체 시즌 대상)...")

    # Feature: Apex Speed
    X = df[['ApexSpeed']]

    # 4개 클러스터로 분류
    kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
    df['Cluster'] = kmeans.fit_predict(X)

    # 속도 순으로 클러스터 ID 정렬 (0: 저속 -> 3: 초고속)
    centers = kmeans.cluster_centers_.flatten()
    sorted_idx = np.argsort(centers)
    mapping = {old: new for new, old in enumerate(sorted_idx)}

    df['Cluster_Label'] = df['Cluster'].map(mapping)

    # 라벨링
    labels = {
        0: 'Low Speed (헤어핀)',
        1: 'Medium Speed (중속)',
        2: 'High Speed (고속)',
        3: 'Ultra High (초고속)'
    }
    df['Corner_Type'] = df['Cluster_Label'].map(labels)

    print("✅ 클러스터링 완료.")
    return df

In [7]:
# 클러스터링 실행
if not df_all_corners.empty:
    df_clustered = perform_clustering(df_all_corners)

    # 분류 결과 확인 (각 유형별 개수)
    print("\n📊 코너 유형별 분포:")
    print(df_clustered['Corner_Type'].value_counts())


🤖 K-Means Clustering 수행 중 (전체 시즌 대상)...
✅ 클러스터링 완료.

📊 코너 유형별 분포:
Corner_Type
Low Speed (헤어핀)      5662
Medium Speed (중속)    4076
High Speed (고속)      2577
Ultra High (초고속)     1024
Name: count, dtype: int64


In [11]:
# 공통 색상 맵 정의
COLOR_MAP = {
    'Low Speed (헤어핀)': '#EF553B',    # Red
    'Medium Speed (중속)': '#FFA15A',   # Orange
    'High Speed (고속)': '#00CC96',     # Green
    'Ultra High (초고속)': '#636EFA'    # Blue
}

def visualize_circuit_dna(df):
    """Chart 1: 서킷별 코너 DNA 분포 (Strip Plot)"""
    if df.empty:
        print("⚠️ 시각화할 데이터가 없습니다.")
        return

    print("🎨 서킷 코너링 DNA 지도 생성 중...")

    # 2024년 최신 데이터 기준
    latest_df = df[df['Year'] == 2024]

    fig = px.strip(
        latest_df,
        x="ApexSpeed",
        y="Circuit",
        color="Corner_Type",
        hover_data=["CornerID", "ApexSpeed"],
        title="🏁 [2024 기준] 서킷별 코너링 DNA 지도 (Corner DNA Map)",
        color_discrete_map=COLOR_MAP,
        category_orders={"Corner_Type": ['Low Speed (헤어핀)', 'Medium Speed (중속)', 'High Speed (고속)', 'Ultra High (초고속)']}
    )
    fig.update_layout(
        template="plotly_dark",
        height=800,
        xaxis_title="코너 최저 속도 (km/h)",
        yaxis_title="서킷"
    )
    fig.show()

def visualize_corner_evolution(df):
    """Chart 2: 연도별 코너 속도 변화 (Box Plot)"""
    if df.empty:
        print("⚠️ 시각화할 데이터가 없습니다.")
        return

    print("🎨 코너링 속도 진화 차트 생성 중...")

    # 규정 변화가 코너링 속도에 미친 영향 확인
    fig = px.box(
        df,
        x="Year",
        y="ApexSpeed",
        color="Corner_Type",
        title="📉 F1 규정 변화와 코너링 속도의 진화 (2018-2025)",
        color_discrete_map=COLOR_MAP,
        category_orders={"Corner_Type": ['Low Speed (헤어핀)', 'Medium Speed (중속)', 'High Speed (고속)', 'Ultra High (초고속)']}
    )
    fig.update_layout(
        template="plotly_dark",
        height=600,
        xaxis_title="연도 (Year)",
        yaxis_title="코너 최저 속도 (km/h)"
    )

    # 2022년 규정 대격변 표시선
    fig.add_vline(x=2022, line_dash="dash", line_color="white", annotation_text="Ground Effect Era")

    fig.show()

In [12]:
if not df_clustered.empty:
    visualize_circuit_dna(df_clustered)

🎨 서킷 코너링 DNA 지도 생성 중...


**서킷별 코너링 DNA 지도 (Circuit Corner DNA)**
- K-Means 클러스터링을 통한 서킷 성향 분류
- F1 캘린더에 존재하는 서킷들의 수천 개의 코너를 **최적 속도(Apex Speed)** 기준으로 비지도 학습(K-Means, K=4)을 수행하여 4가지 클래스(Low, Medium, High, Ultra High)로 분류
<br/>

- 모나코 vs 제다/라스베가스
    - 모나코(Monaco): 그래프 하단의 모나코는 주황색(Low/Medium) 점들이 빼곡히 있어 지배적입니다. 이는 저속/중속 코너가 많아 평균 속도가 현저히 낮고 기계적 그립이 중요한 서킷임을 증명합니다.
    - 제다(Jeddah) & 라스베가스(Las Vegas): 반면 상단의 제다와 라스베가스는 파란색 점들이 다수 분포합니다. 이는 '스트리트 서킷'임에도 불구하고 공기 역학적 효율성(Aerodynamic Efficiency)이 승패를 가르는 트랙임을 보여줍니다.
- 밸런스의 스즈카
    - 스즈카는 모든 색의 점이 가장 고르게 분포된 서킷 중 하나입니다. 이는 차량의 종합적인 밸런스(Overall Balance)를 극한으로 테스트하는 서킷이라는 명성을 데이터로 뒷받침합니다.

In [13]:
if not df_clustered.empty:
    visualize_corner_evolution(df_clustered)

🎨 코너링 속도 진화 차트 생성 중...


**규정 변화와 코너링 속도의 진화 (2018~2025)**
- 2018년부터 2025년까지 코너링 속도의 변화를 추적하여 2022년 그라운드 이펙트(Ground Effect) 규정 도입이 차량의 거동에 미친 영향을 정량화했습니다.
<br/>

- 저속 코너(Red)의 불변성
    - 그래프 하단의 빨간색 박스(Low Speed)는 7년간 거의 변화가 없습니다.
    - 100km/h 이하의 저속 코너에서는 다운포스(공기역학)가 거의 발생하지 않습니다. 즉, 규정이 바뀌어 윙 모양이 변하더라도, 저속 코너 성능은 오직 타이어와 서스펜션에 의존한다는 차량 동역학의 원리를 데이터로 증명하였습니다.
- 초고속 코너(Blue)의 급락
    - 파란색 박스(Ultra High)를 보면, 2020~2021년 정점을 찍던 속도가 2022년에 급격히 하락(중위값 약 15~20km/h 감소)했습니다.
    - 2022년 규정으로 차량 무게가 증가하고, 그라운드 이펙트 특유의 포포이징(Porpoising)(차가 위아래로 심하게 출렁이는 현상) 문제를 해결하기 위해 차고(Ride Height)를 높이면서 고속 다운포스 손실이 발생했기 때문입니다.
- J커브 회복 (2023~2025)
    - 2023년부터 파란색 박스가 다시 상승하는 추세를 보입니다.
    - 이는 엔지니어들이 새로운 규정에 적응하여, 잃어버렸던 다운포스를 빠르게 회복하고 있음을 보여줍니다.